# 推荐系统样例

## 0/1型数据样例

In [1]:
users = ["User1","User2","User3","User4","User5"]

In [2]:
items = ["Item A","Item B","Item C","Item D","Item E"]

In [3]:
# 用户购买记录数据集
datasets = [
    [1,0,1,1,0],
    [1,0,0,1,1],
    [1,0,1,0,0],
    [0,1,0,1,1],
    [1,1,1,0,1]
]

In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.DataFrame(datasets,columns=items,index=users)

In [6]:
df

,Item A,Item B,Item C,Item D,Item E
User1,1,0,1,1,0
User2,1,0,0,1,1
User3,1,0,1,0,0
User4,0,1,0,1,1
User5,1,1,1,0,1


In [7]:
df.T

,User1,User2,User3,User4,User5
Item A,1,1,1,0,1
Item B,0,0,0,1,1
Item C,1,0,1,0,1
Item D,1,1,0,1,0
Item E,0,1,0,1,1


In [8]:
from sklearn.metrics import jaccard_score

In [9]:
jaccard_score(df["Item A"],df["Item B"])

0.2

In [10]:
from sklearn.metrics.pairwise import pairwise_distances

In [11]:
user_similar = 1 - pairwise_distances(df.values, metric="jaccard")

d:\install\python37\lib\site-packages\sklearn\metrics\pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [12]:
user_similar = pd.DataFrame(user_similar, columns=users, index=users)
print("用户之间的两两相似度：")
print(user_similar)

用户之间的两两相似度：
          User1  User2     User3  User4  User5
User1  1.000000   0.50  0.666667    0.2    0.4
User2  0.500000   1.00  0.250000    0.5    0.4
User3  0.666667   0.25  1.000000    0.0    0.5
User4  0.200000   0.50  0.000000    1.0    0.4
User5  0.400000   0.40  0.500000    0.4    1.0


### 基于用户的 协同过滤-User-Based CF  

**因为df的每一行 是一条完整的数据，所以计算每一行数据之间的 两两相似度**

In [13]:
topN_users = {}
# 遍历每一行数据
for i in user_similar.index:
    # 取出每一列数据，并删除自身，然后排序数据
    _df = user_similar.loc[i].drop([i])
    _df_sorted = _df.sort_values(ascending=False)

    top2 = list(_df_sorted.index[:2])
    topN_users[i] = top2

print("Top2相似用户：")
print(topN_users)

Top2相似用户：
{'User1': ['User3', 'User2'], 'User2': ['User1', 'User4'], 'User3': ['User1', 'User5'], 'User4': ['User2', 'User5'], 'User5': ['User3', 'User1']}


In [14]:
rs_results = {}
# 构建推荐结果
for user, sim_users in topN_users.items():
    rs_result = set()    # 存储推荐结果
    for sim_user in sim_users:
        # 构建初始的推荐结果
        rs_result = rs_result.union(set(df.loc[sim_user].replace(0,np.nan).dropna().index))
    # 过滤掉已经购买过的物品
    # print(rs_result) #  相似用户 购买的物品
    # print(set(df.loc[user].replace(0,np.nan).dropna().index))  # 用户 够买的物品
    rs_result -= set(df.loc[user].replace(0,np.nan).dropna().index)
    # print(rs_result) # 去掉 自己够买的物品  = 推荐的物品
    # assert 0
    rs_results[user] = rs_result
print("最终推荐结果：")
print(rs_results)

最终推荐结果：
{'User1': {'Item E'}, 'User2': {'Item B', 'Item C'}, 'User3': {'Item E', 'Item B', 'Item D'}, 'User4': {'Item A', 'Item C'}, 'User5': {'Item D'}}


### 基于 物品的 协同过滤-Item-Based CF

In [15]:
# 计算物品间相似度
item_similar = 1 - pairwise_distances(df.T.values, metric="jaccard")
item_similar = pd.DataFrame(item_similar, columns=items, index=items)
print("物品之间的两两相似度：")
print(item_similar)

物品之间的两两相似度：
        Item A    Item B  Item C  Item D    Item E
Item A    1.00  0.200000    0.75    0.40  0.400000
Item B    0.20  1.000000    0.25    0.25  0.666667
Item C    0.75  0.250000    1.00    0.20  0.200000
Item D    0.40  0.250000    0.20    1.00  0.500000
Item E    0.40  0.666667    0.20    0.50  1.000000


d:\install\python37\lib\site-packages\sklearn\metrics\pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [16]:
topN_items = {}
# 遍历每一行数据
for i in item_similar.index:
    # 取出每一列数据，并删除自身，然后排序数据
    _df = item_similar.loc[i].drop([i])
    _df_sorted = _df.sort_values(ascending=False)

    top2 = list(_df_sorted.index[:2])
    topN_items[i] = top2
print("Top2相似物品：")
print(topN_items)

Top2相似物品：
{'Item A': ['Item C', 'Item D'], 'Item B': ['Item E', 'Item C'], 'Item C': ['Item A', 'Item B'], 'Item D': ['Item E', 'Item A'], 'Item E': ['Item B', 'Item D']}


In [20]:
rs_results = {}
# 构建推荐结果
for user in df.index:    # 遍历所有用户
    rs_result = set()
    for item in df.loc[user].replace(0,np.nan).dropna().index:   # 取出每个用户当前已购物品列表
        # 根据每个物品找出最相似的TOP-N物品，构建初始推荐结果
        rs_result = rs_result.union(topN_items[item])  # 找到与物品A 相似 的相关物品
    print(user)
    print(rs_result)
    print(set(df.loc[user].replace(0,np.nan).dropna().index))
    # 过滤掉用户已购的物品
    rs_result -= set(df.loc[user].replace(0,np.nan).dropna().index)
    print(rs_result)
    print("*"*10)
    # assert 0
    # 添加到结果中
    rs_results[user] = rs_result

print("最终推荐结果：")
print(rs_results)

User1
{'Item E', 'Item B', 'Item A', 'Item D', 'Item C'}
{'Item A', 'Item D', 'Item C'}
{'Item E', 'Item B'}
**********
User2
{'Item E', 'Item B', 'Item A', 'Item D', 'Item C'}
{'Item E', 'Item A', 'Item D'}
{'Item B', 'Item C'}
**********
User3
{'Item B', 'Item A', 'Item D', 'Item C'}
{'Item A', 'Item C'}
{'Item B', 'Item D'}
**********
User4
{'Item E', 'Item B', 'Item A', 'Item D', 'Item C'}
{'Item B', 'Item E', 'Item D'}
{'Item A', 'Item C'}
**********
User5
{'Item E', 'Item B', 'Item A', 'Item D', 'Item C'}
{'Item B', 'Item E', 'Item A', 'Item C'}
{'Item D'}
**********
最终推荐结果：
{'User1': {'Item E', 'Item B'}, 'User2': {'Item B', 'Item C'}, 'User3': {'Item B', 'Item D'}, 'User4': {'Item A', 'Item C'}, 'User5': {'Item D'}}


## 评分型/连续型数据样例

**使用协同过滤推荐算法对用户进行评分预测** 
</br>
* 构建数据集：对于缺失的部分我们需要保留为None，如果设置为0那么会被当作评分值为0去对待

In [21]:
users = ["User1", "User2", "User3", "User4", "User5"]
items = ["Item A", "Item B", "Item C", "Item D", "Item E"]
# 用户购买记录数据集
datasets = [
    [5,3,4,4,None],
    [3,1,2,3,3],
    [4,3,4,3,5],
    [3,3,1,5,4],
    [1,5,5,2,1],
]

**计算相似度：对于评分数据这里我们采用皮尔逊相关系数[-1,1]来计算，-1表示强负相关，+1表示强正相关**
</br>
* pandas中corr方法可直接用于计算皮尔逊相关系数

In [22]:
df = pd.DataFrame(datasets,
                  columns=items,
                  index=users)
print(df)
print("用户之间的两两相似度：")
# 直接计算皮尔逊相关系数
# 默认是按列进行计算，因此如果计算用户间的相似度，当前需要进行转置
user_similar = df.T.corr()
print(user_similar.round(4))

print("物品之间的两两相似度：")
item_similar = df.corr()
print(item_similar.round(4))

       Item A  Item B  Item C  Item D  Item E
User1       5       3       4       4     NaN
User2       3       1       2       3     3.0
User3       4       3       4       3     5.0
User4       3       3       1       5     4.0
User5       1       5       5       2     1.0
用户之间的两两相似度：
        User1   User2   User3   User4   User5
User1  1.0000  0.8528  0.7071  0.0000 -0.7921
User2  0.8528  1.0000  0.4677  0.4900 -0.9001
User3  0.7071  0.4677  1.0000 -0.1612 -0.4666
User4  0.0000  0.4900 -0.1612  1.0000 -0.6415
User5 -0.7921 -0.9001 -0.4666 -0.6415  1.0000
物品之间的两两相似度：
        Item A  Item B  Item C  Item D  Item E
Item A  1.0000 -0.4767 -0.1231  0.5322  0.9695
Item B -0.4767  1.0000  0.6455 -0.3101 -0.4781
Item C -0.1231  0.6455  1.0000 -0.7206 -0.4276
Item D  0.5322 -0.3101 -0.7206  1.0000  0.5817
Item E  0.9695 -0.4781 -0.4276  0.5817  1.0000


可以看到与用户1最相似的是用户2和用户3；与物品A最相似的物品分别是物品E和物品D。

**注意:** 我们在预测评分时，往往是通过与其有正相关的用户或物品进行预测，如果不存在正相关的情况，那么将无法做出预测。这一点尤其是在稀疏评分矩阵中尤为常见，因为稀疏评分矩阵中很难得出正相关系数。

**评分预测:**
User-Based CF 评分预测：使用用户间的相似度进行预测

关于评分预测的方法也有比较多的方案，下面介绍一种效果比较好的方案，该方案考虑了用户本身的评分以及近邻用户的加权平均相似度打分来进行预测：

url: https://xfliu1998.github.io/2022/01/18/5.1-Recommendation-System-Introduction/

Item-Based CF 评分预测：使用物品间的相似度进行预测

这里利用物品相似度预测的计算同上，同样考虑了用户自身的平均打分因素，结合预测物品与相似物品的加权平均相似度打分进行来进行预测

url: https://xfliu1998.github.io/2022/01/18/5.1-Recommendation-System-Introduction/